In [1]:
import os
import os.path
import time
import pandas as pd
import numpy as np
import pathlib
from io import StringIO
from Bio import SeqIO
from Bio.Cluster import distancematrix
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import subprocess

Make sure blast is on path

in .bashrc add

export PATH="$PATH:~/somefolder/ncbi-blast-2.11.0+/bin"

before running this notebook run:

export BLASTDB=/somepath/database_directory


In [2]:
PRJ='PRJNA607174'
PRJ_PATH=f'/mnt/6TB_0/Data/genbank/{PRJ}/'
DATA_PATH=f'/mnt/1TB_0/Data/Assembly/{PRJ}/'

DB='nt'

#percentage identity 
PER_ID = 95
#Expect value (E) for saving hits
E_VAL=0.001

RUN_TYPE='soft_clipped'

SOFT_FILE_POSTFIX='_soft_reads_GD_1_soft_clip_bwamem2_gatk_sorted_marked.fa'
NUM_THREADS=6

Export a accession from db

In [3]:
def get_reads(sra_file):
    readlist=[]
    idlist=[]
    for record in SeqIO.parse(sra_file, "fasta"):
        idlist.append(record.id)
        readlist.append(record.seq)
    return idlist, readlist

### Local Blast

In [4]:
#local BLAST
#see https://www.biostars.org/p/332113/
def search(idx, query_string, database_path, alfile, hspfile):
    start_time=time.time()

    #Number of aligned sequences to keep. 
    max_tgt_seqs=1
    #Maximum number of HSPs (alignments) to keep for any single query-subject pair.
    #If this option is not set (None), BLAST shows all HSPs meeting the expect value criteria.
    max_hsps=1
    blastn_cline = NcbiblastnCommandline(
                                         db=database_path, 
                                         evalue=E_VAL,
                                         outfmt=5, 
                                         perc_identity=PER_ID,
                                         max_target_seqs=max_tgt_seqs, 
                                         max_hsps=max_hsps,
                                         num_threads=NUM_THREADS
                                        )
    out, err = blastn_cline(stdin=query_string)
    io_result = StringIO(out)
    blast_records = list(NCBIXML.parse(io_result))
    for blast_record in blast_records:
        if len(blast_record.alignments) == 0:
            continue
        else:
            alignment = blast_record.alignments[0]
            title = alignment.title
            query_length = blast_record.query_letters
            alfile.write(f"id: {idx}, title: {title}, accession: {alignment.accession}\n")
            for hsp in alignment.hsps:
                hspfile.write(f"id: {idx}, title: {title}, accession: {alignment.accession}, hit_id: {alignment.hit_id}, length: {alignment.length}, query_length {blast_record.query_letters}, score: {hsp.score}, expect: {hsp.expect}, align_length: {hsp.align_length}, bits: {hsp.bits}, query: {hsp.query}, sbjct: {hsp.sbjct}, query_start: {hsp.query_start}, query_end: {hsp.query_end}, sbjct_start: {hsp.sbjct_start}, sbjct_end: {hsp.sbjct_end}\n")
    return 

In [5]:
def process_blast(sra_fasta_path, blast_path, fasta):
    idlist, readlist=get_reads(sra_fasta_path)

    fast_f=fasta.split('.fa')[0]
    result_file=f'{blast_path}{fast_f}_blast_{DB}_PCT{PER_ID}_E{E_VAL}.csv'
    hsp_file=f'{blast_path}{fast_f}_blast_{DB}_PCT{PER_ID}_E{E_VAL}_hsps.txt'
    hit_data=[]
    
    resultf= open(result_file,"a")
    hspf= open(hsp_file,"a")
    start_time=time.time()
    i=0
    for idx, r in zip(idlist,readlist):
        hit_data.append(search(idx, r, DB, resultf, hspf))
    resultf.close()
    hspf.close()

In [6]:
def run_blast(sra, fasta_path, fasta):
    
    blast_path=DATA_PATH+sra+'/'
    blast_path=blast_path+'blast/'
    pathlib.Path(blast_path).mkdir(exist_ok=True)
    
    sra_fasta_path=fasta_path+fasta
    #empty if no alignments
    with open(sra_fasta_path,"r") as f:
        if len(f.readlines())>0:
            print(f'--run_blast on sra: {sra}, fasta: {fasta}')
            process_blast(sra_fasta_path, blast_path, fasta)

In [7]:

SRAs=['SRR13053879']
BAM_POST='_gd1_amplicon_seq_GD_1_soft_clip_bwamem2_gatk_sorted_marked.bam'

In [8]:
REF='GD_1'

PREFIXES=['ml15__start','ml15__end']

In [9]:
idx=0
for sra in SRAs:
    for pref in PREFIXES:
        fasta=sra+'_'+REF+'_'+pref+SOFT_FILE_POSTFIX
        fasta_path=DATA_PATH+sra+'/bwa_mem2/soft_clip/'
        if not os.path.isfile(fasta_path+fasta):
            print(fasta_path+fasta) 
            raise Exception

        run_blast(sra,fasta_path,fasta)


--run_blast on sra: SRR13053879, fasta: SRR13053879_GD_1_ml15__start_soft_reads_GD_1_soft_clip_bwamem2_gatk_sorted_marked.fa
--run_blast on sra: SRR13053879, fasta: SRR13053879_GD_1_ml15__end_soft_reads_GD_1_soft_clip_bwamem2_gatk_sorted_marked.fa
